In [24]:
tezheng1.columns

Index([u'fengsu', u'fengsu_dif', u'file_name'], dtype='object')

In [22]:
import pandas as pd
all=pd.read_csv("all_0810_v3.csv")
tezheng1 = pd.read_csv('tezheng_train.csv')
all.columns

Index([u'f_id', u'file_name', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7',
       ...
       u'516', u'517', u'518', u'519', u'520', u'521', u'522', u'523', u'524',
       u'ret'],
      dtype='object', length=528)

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV

In [26]:
x1,x2, y = all.iloc[:, 2:227],all.iloc[:, 302:377],all.iloc[:,len(all.columns)-1:]
x = (x1.T.append(x2.T)).T
x = (x.T.append(tezheng1.iloc[:,:2].T)).T
x.shape,y.columns

((48339, 302), Index([u'ret'], dtype='object'))

In [4]:
y.sum()

In [5]:
# x.columns,x.index
all.shape

(48339, 303)

In [27]:
x = StandardScaler().fit_transform(x)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5)

In [29]:
def lightGBM(train_x,train_y,validation_x,validation_y):
    clf = lgb.LGBMClassifier(num_leaves= 63, 
                             max_depth=5, 
                             silent=True,
                             metric='auc',
                             n_jobs=8,
                             n_estimators=1000,
                             colsample_bytree=0.9,
                             subsample=0.9,
                             learning_rate=0.35
                            )
#     clf =lgb.LGBMClassifier(boosting_type='gbdt',
#                              num_class=2, 
#                              num_leaves=55, 
#                              reg_lambda=1,
#                              max_depth=12, 
#                              n_estimators=1000, 
#                              objective='binary',
#                              subsample=0.8, 
#                              colsample_bytree=0.8, 
#                              subsample_freq=1,
#                              learning_rate=0.01, 
#                              min_child_weight=1, 
#                              random_state=20, 
#                              n_jobs=4
#                             )
    
    # clf.fit(train_x, train_y, **fit_params)
    clf.fit(train_x,
    train_y,
    early_stopping_rounds=200,
    eval_metric= 'auc', 
            eval_set = [(validation_x,validation_y)],
            eval_names = ['valid'],
            verbose= 100
    )
         
    return clf

clf = lightGBM(x_train,y_train, x_test, y_test)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
print u'lgb训练集准确率：', f1_score(y_train, y_train_pred)
print u'lgb测试集准确率：', f1_score(y_test, y_test_pred)

F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 200 rounds.
[100]	valid's auc: 0.991168
[200]	valid's auc: 0.996122
[300]	valid's auc: 0.997259
[400]	valid's auc: 0.997786
[500]	valid's auc: 0.997985
[600]	valid's auc: 0.998151
[700]	valid's auc: 0.998259
[800]	valid's auc: 0.998322
[900]	valid's auc: 0.998409
[1000]	valid's auc: 0.998443
Did not meet early stopping. Best iteration is:
[964]	valid's auc: 0.998454


F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


lgb训练集准确率： 0.9999152685985426
lgb测试集准确率： 0.983398151633003


F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [63]:
y_train.ret.values#,x_train.shape

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [8]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(x_train, y_train.T.values[0])
lgb_eval = lgb.Dataset(x_test, y_test.T.values[0], reference=lgb_train)
lgb_cv = lgb.Dataset(x, y.T.values[0])
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 5,
    'max_depth': 6,
    'min_data_in_leaf': 400,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5,
    'lambda_l1': 1,  
    'lambda_l2': 0.001,  # 越小l2正则程度越高
    'min_gain_to_split': 0.2,
    'is_unbalance': True
}

# train
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=lgb_eval,
                early_stopping_rounds=200,
                verbose_eval = 100
               )
# gbm = lgb.cv(params,
#              lgb_cv,
#              num_boost_round=1000,
#              nfold = 4,
#              early_stopping_rounds=200,
#              verbose_eval = 100
#                )

print('Start predicting...')

preds = gbm.predict(x_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
# 导出结果
# threshold = 0.5
# for pred in preds:
#     result = 1 if pred > threshold else 0

# # 导出特征重要性
# importance = gbm.feature_importance()
# names = gbm.feature_name()

Start training...
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.935327	valid_0's binary_logloss: 0.376774
[200]	valid_0's auc: 0.960798	valid_0's binary_logloss: 0.301342
[300]	valid_0's auc: 0.972288	valid_0's binary_logloss: 0.255556
[400]	valid_0's auc: 0.977279	valid_0's binary_logloss: 0.230548
[500]	valid_0's auc: 0.981568	valid_0's binary_logloss: 0.208647
[600]	valid_0's auc: 0.983975	valid_0's binary_logloss: 0.193845
[700]	valid_0's auc: 0.985788	valid_0's binary_logloss: 0.180236
[800]	valid_0's auc: 0.986879	valid_0's binary_logloss: 0.171974
[900]	valid_0's auc: 0.987583	valid_0's binary_logloss: 0.165002
[1000]	valid_0's auc: 0.988644	valid_0's binary_logloss: 0.156833
[1100]	valid_0's auc: 0.989549	valid_0's binary_logloss: 0.15002
[1200]	valid_0's auc: 0.989919	valid_0's binary_logloss: 0.145617
[1300]	valid_0's auc: 0.990435	valid_0's binary_logloss: 0.140401
[1400]	valid_0's auc: 0.990869	valid_0's binary_logloss: 0.136446
[1500

In [30]:
rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, oob_score=True)
rf.fit(x_train, y_train)
print u'OOB Score=%.5f' % rf.oob_score_
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)
print u'随机森林训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'随机森林测试集准确率：', accuracy_score(y_test, y_test_pred)
print u'随机森林测试集f1_score：', f1_score(y_test, y_test_pred)

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
gb.fit(x_train, y_train)
y_train_pred = gb.predict(x_train)
y_test_pred = gb.predict(x_test)
print u'GBDT训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'GBDT测试集准确率：', accuracy_score(y_test, y_test_pred)
print u'GBDT测试集f1_score：', f1_score(y_test, y_test_pred)

F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


OOB Score=0.83806
随机森林训练集准确率： 0.8536968844387438
随机森林测试集准确率： 0.8417873396772859
随机森林测试集f1_score： 0.8477222045237336
GBDT训练集准确率： 0.9363233894658447
GBDT测试集准确率： 0.9145635084815887
GBDT测试集f1_score： 0.9147996864298388


In [9]:
train_test_pred = pd.DataFrame(preds)
train_test_pred.columns = ['Lable']
train_test_pred['Lable'] =train_test_pred.apply(lambda x: 1 if(x['Lable']>0.53) else 0,axis=1)
print u'f1_score：', f1_score(y_test, train_test_pred.Lable)
print u'accuracy_score：', accuracy_score(y_test, train_test_pred.Lable)
print u'precision_score：', precision_score(y_test, train_test_pred.Lable)
# list(preds)

f1_score： 0.9666892561121826
accuracy_score： 0.9669528340918494
precision_score： 0.9578512396694215


In [24]:
all_1 = [1]*40000+[0]*8339
all_1 = pd.DataFrame(all_1)
print u'f1_score：', f1_score(y, all_1)

f1_score： 0.611157368859284


In [31]:
test = pd.read_csv('test_0810.csv')
tezheng2 = pd.read_csv('tezheng_test.csv')
test_data = test.iloc[:, 1:len(test.columns)-1]
t1,t2 = test_data.iloc[:, :225],test_data.iloc[:, 300:375]
t = (t1.T.append(t2.T)).T
test_data = (t.T.append(tezheng2.iloc[:,:2].T)).T
test_data.shape

(87498, 302)

In [26]:
test_data.columns

Index([u'轮毂转速_median', u'轮毂角度_median', u'叶片1角度_median', u'叶片2角度_median',
       u'叶片3角度_median', u'变桨电机1电流_median', u'变桨电机2电流_median',
       u'变桨电机3电流_median', u'超速传感器转速检测值_median', u'5秒偏航对风平均值_median',
       ...
       u'叶片3变频器箱温度_kurt', u'叶片1超级电容电压_kurt', u'叶片2超级电容电压_kurt',
       u'叶片3超级电容电压_kurt', u'驱动1晶闸管温度_kurt', u'驱动2晶闸管温度_kurt', u'驱动3晶闸管温度_kurt',
       u'驱动1输出扭矩_kurt', u'驱动2输出扭矩_kurt', u'驱动3输出扭矩_kurt'],
      dtype='object', length=525)

In [32]:
# test_data = StandardScaler().fit_transform(test_data)
test_data_pred = rf.predict(test_data)  # 输出的是概率结果
test_data_pred = pd.DataFrame(test_data_pred)
test_data_pred.shape

(87498, 1)

In [34]:
test_data_pred.sum()

0    17484
dtype: int64

In [14]:
result_test_v1 = test.iloc[:, :1]
result_test_v1['Lable'] = test_data_pred[0]
# result_test_v1['Lable'] =result_test_v1.apply(lambda x: 1 if(x['Lable']>0.68387) else 0,axis=1)
result_test_v1.rename(columns={'file_name':'ID'}, inplace = True)
print result_test_v1.head(10)
#print result_test_v1.sum()
result_test_v1.to_csv('result_0819.csv',index = None)

                                         ID  Lable
0  ffff6094-6361-302d-abbe-f6a0dbb771d6.csv      1
1  fffecc43-bb76-3ad2-8eed-86fdd7cb81b2.csv      0
2  fffeae6f-9a72-3193-ba3d-ef0c0bea08f4.csv      1
3  fffe96ae-cdbe-349f-8906-214ab98e8ff8.csv      0
4  fffd5e08-81af-3137-8dbe-4460f1ff39ff.csv      1
5  fffd1921-43fb-3289-b251-3329ae7541f0.csv      0
6  fffd0c63-f9aa-35d8-a7b5-3cddb7238984.csv      0
7  fffcea6d-3f76-364e-afc5-9f40479b6d33.csv      0
8  fffc909f-a24b-3036-957f-6e5290a009f2.csv      0
9  fff7f440-f1af-3800-986c-dcf8eb827944.csv      0


F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [71]:
all_1 = [1]*87498
all_1 = pd.DataFrame(all_1)
print u'f1_score：', f1_score(all_1, result_test_v1.Lable)
result_test_v1.Lable.sum()


f1_score： 0.6650697993744755


43592

In [20]:
result_test_v1 = test.iloc[:, :1]
result_test_v1['Lable'] = test_data_pred[0]
result_test_v1.rename(columns={'file_name':'ID'}, inplace = True)
result_test_v1.head(20)

F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ID,Lable
0,ffff6094-6361-302d-abbe-f6a0dbb771d6.csv,0.823651
1,fffecc43-bb76-3ad2-8eed-86fdd7cb81b2.csv,0.009462
2,fffeae6f-9a72-3193-ba3d-ef0c0bea08f4.csv,0.882495
3,fffe96ae-cdbe-349f-8906-214ab98e8ff8.csv,0.005902
4,fffd5e08-81af-3137-8dbe-4460f1ff39ff.csv,0.869751
5,fffd1921-43fb-3289-b251-3329ae7541f0.csv,0.726151
6,fffd0c63-f9aa-35d8-a7b5-3cddb7238984.csv,0.001736
7,fffcea6d-3f76-364e-afc5-9f40479b6d33.csv,0.680423
8,fffc909f-a24b-3036-957f-6e5290a009f2.csv,0.819215
9,fff7f440-f1af-3800-986c-dcf8eb827944.csv,0.004627


In [11]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data=pd.read_csv("all_result_v2_0715.csv")

x_prime = data.iloc[0:,1:len(data.columns)-1] #去掉y列，所以len-1
y = data.iloc[0:,len(data.columns)-1:]
x_prime_train, x_prime_test, y_train, y_test = train_test_split(x_prime, y, train_size=0.7, random_state=0)
feature_pairs=[]

for p in range(len(data.columns)-1):
    # 准备数据
    x_train = x_prime_train.iloc[0:,range(p)+range(p+1,len(data.columns)-2)]#去掉y列之外再去掉一列特征，所以len-2
    x_test = x_prime_test.iloc[0:,range(p)+range(p+1,len(data.columns)-2)]

    # 决策树学习
    model = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)
    model.fit(x_train, y_train)

    # 训练集上的预测结果
    y_train_pred = model.predict(x_train)
    acc_train = accuracy_score(y_train, y_train_pred)
    y_test_pred = model.predict(x_test)
    acc_test = accuracy_score(y_test, y_test_pred)
    if acc_train > 0.85 and acc_test > 0.7:
        print '不要特征：', data.columns[p]
        print '\t训练集准确率: %.4f%%' % (100*acc_train)
        print '\t测试集准确率: %.4f%%\n' % (100*acc_test)

不要特征： fengjiid
	训练集准确率: 99.8258%
	测试集准确率: 99.6614%

不要特征： 轮毂转速_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7630%

不要特征： 轮毂角度_median
	训练集准确率: 99.8258%
	测试集准确率: 99.5936%

不要特征： 叶片1角度_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7968%

不要特征： 叶片2角度_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6614%

不要特征： 叶片3角度_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6275%

不要特征： 变桨电机1电流_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6952%

不要特征： 变桨电机2电流_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6952%

不要特征： 变桨电机3电流_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6275%

不要特征： 超速传感器转速检测值_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6614%

不要特征： 5秒偏航对风平均值_median
	训练集准确率: 99.8258%
	测试集准确率: 99.3227%

不要特征： x方向振动值_median
	训练集准确率: 99.8403%
	测试集准确率: 99.4582%

不要特征： y方向振动值_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 扭缆圈数_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 液压制动压力_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 机舱气象站风速_median
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 风向绝对值_median
	训练集准确率: 99.8258%
	测试集准确率: 99.6275%

不要特征： 大气压力_median
	训练集准确率: 99.8258%

不要特征： 叶片2超级电容电压_quantile25
	训练集准确率: 99.8403%
	测试集准确率: 99.6275%

不要特征： 叶片3超级电容电压_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.6952%

不要特征： 驱动1晶闸管温度_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.7630%

不要特征： 驱动2晶闸管温度_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.6275%

不要特征： 驱动3晶闸管温度_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.5936%

不要特征： 驱动1输出扭矩_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.6614%

不要特征： 驱动2输出扭矩_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.6275%

不要特征： 驱动3输出扭矩_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 变频器电网侧Cosphi_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.5598%

不要特征： 电网频率误差_quantile25
	训练集准确率: 99.8258%
	测试集准确率: 99.5598%

不要特征： 轮毂转速_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.7968%

不要特征： 轮毂角度_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.7630%

不要特征： 叶片1角度_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.6952%

不要特征： 叶片2角度_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.7291%

不要特征： 叶片3角度_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.6614%

不要特征： 变桨电机1电流_quantile75
	训练集准确率: 99.8258%
	测试集准确率: 99.6952%

不要特征： 变桨电

<Figure size 792x576 with 0 Axes>